In [15]:
# Obesity Prediction Experiment
# Versão corrigida com alinhamento total de features

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

# Carregar dataset
df = pd.read_csv('../data/ObesityDataSet.csv')

# Pré-processamento
def preprocess_data(df):
    # Mapeamentos binários
    df['family_history_with_overweight'] = df['family_history_with_overweight'].map({'yes': 1, 'no': 0})
    df['FAVC'] = df['FAVC'].map({'yes': 1, 'no': 0})
    df['SMOKE'] = df['SMOKE'].map({'yes': 1, 'no': 0})
    df['SCC'] = df['SCC'].map({'yes': 1, 'no': 0})
    df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
    
    # Mapeamentos multi-classe
    df['CAEC'] = df['CAEC'].map({'Never': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
    df['CALC'] = df['CALC'].map({'Never': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
    df['MTRANS'] = df['MTRANS'].map({
        'Automobile': 0, 
        'Bike': 1, 
        'Motorbike': 2, 
        'Public_Transportation': 3, 
        'Walking': 4
    })
    
    # Remoção de NA
    df = df.dropna(subset=['CALC'])
    
    return df

# Aplicar pré-processamento
df_processed = preprocess_data(df.copy())

# Verificar colunas processadas
print("Colunas disponíveis após pré-processamento:")
print(df_processed.columns.tolist())

# Definir features usadas no modelo (DEVE SER IGUAL NO PREPARE_INPUT)
FEATURES = [
    'Age',
    'Gender',
    'Height', 
    'Weight',
    'FAF',
    'SMOKE',
    'FAVC',
    'family_history_with_overweight',
    'CAEC',
    'CALC',
    'MTRANS'
]

# Separar features e target
X = df_processed[FEATURES]  # Usar apenas as features definidas
y = df_processed['NObeyesdad']

# Codificar o target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Verificar mapeamento de classes
print("\nMapeamento de classes para predição:")
class_mapping = {i: cls for i, cls in enumerate(label_encoder.classes_)}
print(class_mapping)

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42
)

# Treinar modelo
model = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

model.fit(X_train, y_train)

# Avaliar
y_pred = model.predict(X_test)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Função de preparação de dados (DEVE USAR MESMA ORDEM QUE FEATURES)
def prepare_input(data):
    """Prepara dados de entrada na EXATA ordem das features de treino"""
    return np.array([
        data['Age'],
        1 if data['Gender'] == 'Male' else 0,
        float(data['Height']),
        float(data['Weight']),
        int(data['FAF']),
        1 if data['SMOKE'] == 'yes' else 0,
        1 if data['FAVC'] == 'yes' else 0,
        1 if data['family_history_with_overweight'] == 'yes' else 0,
        {'Never': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}[data['CAEC']],
        {'Never': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}[data['CALC']],
        {
            'Automobile': 0, 
            'Bike': 1, 
            'Motorbike': 2, 
            'Public_Transportation': 3, 
            'Walking': 4
        }[data['MTRANS']]
    ]).reshape(1, -1)

# Verificar ordem das features
print("\nOrdem das features esperada pelo modelo:")
print(FEATURES)

# Testar com dados simulados
frontend_data = {
    'Age': 30,
    'Gender': 'Female',
    'Height': 1.65,
    'Weight': 70.5,
    'FAF': 2,
    'SMOKE': 'no',
    'FAVC': 'yes',
    'family_history_with_overweight': 'yes',
    'CAEC': 'Sometimes',
    'CALC': 'Never',
    'MTRANS': 'Public_Transportation'
}

input_data = prepare_input(frontend_data)
print("\nDados preparados para o modelo:")
print(pd.DataFrame(input_data, columns=FEATURES))

prediction = model.predict(input_data)
prediction_label = label_encoder.inverse_transform(prediction)[0]

print(f"\nPredição: {prediction_label}")

# Salvar modelo e encoder
joblib.dump(model, '../models/obesity_rf_model.joblib')
joblib.dump(label_encoder, '../models/label_encoder.joblib')
joblib.dump(FEATURES, '../models/features_order.joblib')  # Salva a ordem das features

print("\nArquivos salvos em:")
print("- models/obesity_rf_model.joblib")
print("- models/label_encoder.joblib")
print("- models/features_order.joblib")

Colunas disponíveis após pré-processamento:
['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad']

Mapeamento de classes para predição:
{0: 'Insufficient_Weight', 1: 'Normal_Weight', 2: 'Obesity_Type_I', 3: 'Obesity_Type_II', 4: 'Obesity_Type_III', 5: 'Overweight_Level_I', 6: 'Overweight_Level_II'}

Relatório de Classificação:
                     precision    recall  f1-score   support

Insufficient_Weight       1.00      0.98      0.99        50
      Normal_Weight       0.80      0.87      0.83        52
     Obesity_Type_I       0.94      0.93      0.93        54
    Obesity_Type_II       0.99      0.99      0.99        69
   Obesity_Type_III       1.00      1.00      1.00        97
 Overweight_Level_I       0.79      0.77      0.78        73
Overweight_Level_II       0.79      0.79      0.79        47

           accuracy                           0.91       442
 

C:\Users\pedri\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y_encoded, cv=5)
print("Acurácia média em validação cruzada:", scores.mean())